### Import Library

In [1]:
import os

from formula import *
import json_function as fjson
import processing as proc
import training_data as dtrain
from graph import *
from post_process import *
import internal_factor as infac

### Define Global Variable

In [2]:
data_path = "E:/_TUGAS/_ITBOneDrive/OneDrive - Institut Teknologi Bandung/_Kuliah/_sem7/7_kerja praktek/data/data training 74hc4051"
folder_name = ["data training/"]
folder_path = folder_name.copy()

fstart = 20e3
fend = 50e3

# Main Program

### Considering Internal Factor

In [3]:
internal_flag = infac.choose_internal_flag()

if internal_flag: infac.get_internal_factor(data_path, file_path="tmp/training_internal_factor.json")


Initialize tmp/training_internal_factor.json ... Done


Writing tmp/training_internal_factor.json ... Done


In [4]:
# start
# replace path_value in folder_path
i = 0
for f in folder_path:
    folder_path[i] = os.path.join(data_path, f)
    i += 1

# prepare tmp files
fjson.initialize_training_tmp_files()

Initialize tmp files ... Done


In [5]:
# process analysis
idx = 0
folder_path_i = folder_path[idx]

print("\nProcessing %s ..." %folder_name[idx])

# preprocessing
files, dfs, dfs_list = proc.prepare_data(folder_path[idx])
if internal_flag:
    dfs_list = proc.create_actual_params_columns(dfs_list, file_path="tmp/training_internal_factor.json")
variation_str, variation_data = dtrain.preprocessing_data_training(files)
saved_dirname = proc.prepare_result_folder(data_path)

iteration = len(dfs_list) // len(variation_data)

# main processing
# process_analysis(folder_path[idx], variation_data, dfs_list, iteration)

print("files: %s\n" %files)
print("variation_str: %s\n" %variation_str)
print("variation_data: %s\n" %variation_data)
print("saved_dirname: %s\n" %saved_dirname)


Processing data training/ ...
files: ['1106ohm (1).csv', '1106ohm (2).csv', '1106ohm (3).csv', '1106ohm (4).csv', '1106ohm (5).csv', '995ohm (1).csv', '995ohm (2).csv', '995ohm (3).csv', '995ohm (4).csv', '995ohm (5).csv']

variation_str: ['1106ohm', '995ohm']

variation_data: [1106.0, 995.0]

saved_dirname: data training 74hc4051/



### Main Processing

In [6]:
def process_analysis(folder_path_i, variation_str, dfs_list, iteration):
    saved_dirname = proc.prepare_result_folder(data_path)

    # plot & save figure
    proc.build_graph_per_variation(variation_str, iteration, dfs_list, folder_path_i, saved_dirname)
    
    
    # get z_mid & phase_mid from every dataframe. data_mid is data at fmid
    # arr = [[...], [...], ...]
    arr_z_mid, arr_phase_mid, dfs_list = proc.get_data_mid(dfs_list, iteration)
    print(arr_z_mid)

    print(arr_phase_mid)
    
    # # every variation have n dataframes. n = num_of_iteration
    # # build single dataframe for every variation by averaging them
    # df_choosen = build_df_choosen(dfs_list, iteration)


    # # plot & save figure
    # build_single_graph_from_df_choosen(df_choosen, variation_str, folder_path_i, saved_dirname, internal_flag)


    # # store data_avg of parameter: z, phase. Stored to variatioin_rc_json
    # arr_z_avg, arr_phase_avg = get_z_phase_avg_from_df_choosen(df_choosen, internal_flag)

    # # calculate r,c from z,phase
    # fmid = find_fmid_from_data_retrieval(dfs_list)
    # arr_r, arr_c = get_rc_value(arr_z_avg, arr_phase_avg, fmid)  

    # # update json file data
    # update_retrieval_overview_json(files, iteration, variation_str)
    # update_retrieval_variation_json(variation_str,
    #                                 arr_z_mid, arr_phase_mid,
    #                                 arr_z_avg, arr_phase_avg,
    #                                 arr_r, arr_c)
    # update_retrieval_body_composition_json(variation_str)

In [7]:
process_analysis(folder_path[idx], variation_str, dfs_list, iteration)

NameError: name 'internal_flag' is not defined